# 1. Imports

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd

In [2]:
from MFLUSolution import LUSolution, LUGASolution

In [3]:
from selection_and_operators.mutation import swap_stages_slots_mutation, inversion_mutation, shuffle_mutation
from selection_and_operators.crossover import cycle_crossover, partially_matched_crossover, swap_column_crossover
from selection_and_operators.selection import ranking_selection, tournament_selection
from library.algorithm import genetic_algorithm

In [4]:
from experiment import run_experiment

# (Espaço para testes, depois é apagado)

In [4]:
import random
repr = []
artists = list(range(35))
random.shuffle(artists)
repr = [artists[i * 7:(i + 1) * 7] for i in range(5)]
repr

[[23, 6, 22, 19, 14, 12, 25],
 [3, 30, 29, 17, 24, 0, 2],
 [13, 10, 28, 34, 27, 11, 1],
 [20, 9, 18, 7, 8, 33, 21],
 [16, 5, 26, 15, 4, 32, 31]]

In [5]:
solution= LUSolution(repr=repr)

In [6]:
solution

[[23, 6, 22, 19, 14, 12, 25], [3, 30, 29, 17, 24, 0, 2], [13, 10, 28, 34, 27, 11, 1], [20, 9, 18, 7, 8, 33, 21], [16, 5, 26, 15, 4, 32, 31]]

In [7]:
solution=LUGASolution(
        crossover_function=cycle_crossover,
        mutation_function=swap_stages_slots_mutation,
        repr=repr
    )

In [8]:
solution

[[23, 6, 22, 19, 14, 12, 25], [3, 30, 29, 17, 24, 0, 2], [13, 10, 28, 34, 27, 11, 1], [20, 9, 18, 7, 8, 33, 21], [16, 5, 26, 15, 4, 32, 31]]

In [9]:
solution.fitness()

0.5641482988630827

In [10]:
solution

[[23, 6, 22, 19, 14, 12, 25], [3, 30, 29, 17, 24, 0, 2], [13, 10, 28, 34, 27, 11, 1], [20, 9, 18, 7, 8, 33, 21], [16, 5, 26, 15, 4, 32, 31]]

In [11]:
solution.mutation(mut_prob=0)

[[23, 6, 22, 19, 14, 12, 25], [3, 30, 29, 17, 24, 0, 2], [13, 10, 28, 34, 27, 11, 1], [20, 9, 18, 7, 8, 33, 21], [16, 5, 26, 15, 4, 32, 31]]

In [6]:
solution_new=LUGASolution(
        crossover_function=partially_matched_crossover,
        mutation_function=swap_stages_slots_mutation)

In [7]:
solution_new

[[6, 31, 5, 22, 9, 19, 3], [13, 33, 11, 26, 2, 17, 10], [27, 7, 18, 12, 15, 34, 16], [14, 29, 30, 24, 0, 23, 21], [32, 20, 28, 4, 8, 25, 1]]

In [9]:
solution_new.crossover(solution_new)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 15, 34, 16, 14, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 15, 34, 16, 14, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
offspring1 after index 0: [6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 15, 34, 16, 14, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
offspring2 after index 0: [6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 15, 34, 16, 14, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


offspring1 after index 1: [6, 31, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 15, 34, 16, 14, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
offspring2 after index 1: [6, 31, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 15, 34, 16, 14, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


offspring1 after index 2: [6, 31, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 15, 34, 16, 14, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
offspring2 after index 2: [6, 31, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 15, 34, 16,

([[6, 31, 5, 22, 9, 19, 3], [13, 33, 11, 26, 2, 17, 10], [27, 7, 18, 12, 15, 34, 16], [14, 29, 30, 24, 0, 23, 21], [32, 20, 28, 4, 8, 25, 1]],
 [[6, 31, 5, 22, 9, 19, 3], [13, 33, 11, 26, 2, 17, 10], [27, 7, 18, 12, 15, 34, 16], [14, 29, 30, 24, 0, 23, 21], [32, 20, 28, 4, 8, 25, 1]])

# 2. Testing different parameters for GA

Baseline parameters:
- POP_SIZE = 50, 
- CROSSOVER_FUNCTION = cycle_crossover,
- MUTATION_FUNCTION = shuffle_mutation,
- NUMBER_OF_TESTS=30,
- MAX_GEN=100,
- S_RANKING_SELECTION=1.5,
- K_TOURNEMENT_SELECTION=5,
- SELECTION_ALGORITHM=tournament_selection,
- XO_PROB=0.9,
- MUT_PROB=0.4,
- MUT_MAX_WINDOW_SIZE=5, 
- ELITISM = True,
- VERBOSE = False

### 1st test - crossover and mutation operators:

Operators to test:

- Mutation: swap_slots_mutation, inversion_mutation, shuffle_mutation
- Crossover: cycle_crossover, partially_matched_crossover, swap_column_crossover

In [5]:
mutation_functions = [swap_stages_slots_mutation, inversion_mutation, shuffle_mutation]
crossover_functions = [partially_matched_crossover, swap_column_crossover]

In [6]:
solution=LUGASolution(
        crossover_function=swap_column_crossover,
        mutation_function=swap_stages_slots_mutation
    )

solution2 =LUGASolution(
        crossover_function=swap_column_crossover,
        mutation_function=swap_stages_slots_mutation
    )

In [9]:
repr1, repr2=solution.crossover(solution2)

col swap: 5
parent1: [[10, 14, 4, 33, 34, 23, 12], [28, 13, 25, 27, 31, 8, 3], [11, 6, 22, 1, 0, 15, 26], [29, 18, 24, 16, 19, 7, 17], [32, 20, 21, 9, 5, 30, 2]]
parent2: [[34, 32, 0, 17, 6, 26, 4], [20, 22, 29, 11, 19, 25, 8], [7, 33, 18, 31, 21, 10, 1], [14, 13, 12, 27, 5, 30, 28], [15, 16, 9, 23, 24, 3, 2]]
offspring1 in the end: [[23, 14, 4, 33, 34, 26, 12], [28, 13, 8, 27, 31, 25, 15], [11, 6, 22, 1, 0, 10, 7], [29, 18, 24, 16, 19, 30, 17], [32, 20, 21, 9, 5, 3, 2]]
 offspring2 in the end[[34, 32, 0, 17, 6, 23, 4], [20, 22, 29, 11, 19, 8, 26], [25, 33, 18, 31, 21, 15, 1], [14, 13, 12, 27, 5, 7, 28], [10, 16, 9, 30, 24, 30, 2]]


ValueError: Matrix contain repeated artists

In [8]:
type(repr1)

MFLUSolution.LUGASolution

In [18]:
all_results=[]

for mut in mutation_functions:
    for xo in crossover_functions:
        experiment_name = f"{mut.__name__}__{xo.__name__}"
        print(f"Running experiment: {experiment_name}")
        
        result = run_experiment(
            EXPERIMENT_NAME=experiment_name,
            MUTATION_FUNCTION=mut,
            CROSSOVER_FUNCTION=xo,
            MAX_GEN=2,

            NUMBER_OF_TESTS=2
        )

        all_results.append(result)

Running experiment: swap_stages_slots_mutation__partially_matched_crossover
Iteration 0 of the genetic algorithm

[None, None, None, 31, 9, 17, 0, 26, 24, 8, 21, 32, 27, 7, 16, 18, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]
[None, None, None, 19, 8, 20, 22, 34, 12, 7, 16, 2, 25, 3, 4, 28, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]
{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34}
{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34}
idx1: 3
idx2: 16
parent1: [[13, 23, 14, 19, 8, 20, 22], [34, 12, 7, 16, 2, 25, 3], [4, 28, 15, 10, 9, 26, 32], [29, 11, 31, 27, 5, 1, 33], [17, 0, 30, 21, 18, 24, 6]]
parent2: [[15, 4, 22, 31, 9, 17, 0], [26, 24, 8, 21, 32, 27, 7], [16, 18, 34, 30, 33, 2, 12], [6

ValueError: Matrix contain repeated artists

In [ ]:
for result in all_results:
    plt.plot(
        result["stats_df"]["Generation"],
        result["fitness_avg"],
        label=result["name"]
    )

plt.xlabel("Generation")
plt.ylabel("Average Fitness")
plt.title("Comparison of Operator Combinations")
plt.legend()
plt.grid()
plt.show()


In [8]:
results=run_experiment(MAX_GEN=5, NUMBER_OF_TESTS=5, EXPERIMENT_NAME='')

Iteration 0 of the genetic algorithm

Final best individual in generation 1: 0.6753023160483705
Final best individual in generation 2: 0.7074906545222309
Final best individual in generation 3: 0.7074906545222309
Final best individual in generation 4: 0.7296867562978662
Final best individual in generation 5: 0.760347242345214
Iteration 1 of the genetic algorithm

Final best individual in generation 1: 0.6843813387423935
Final best individual in generation 2: 0.6936733314015262
Final best individual in generation 3: 0.6936733314015262
Final best individual in generation 4: 0.6949673913452529
Final best individual in generation 5: 0.711532889017676
Iteration 2 of the genetic algorithm

Final best individual in generation 1: 0.6839177050130397
Final best individual in generation 2: 0.7367221236686432
Final best individual in generation 3: 0.7515304860379812
Final best individual in generation 4: 0.7515304860379812
Final best individual in generation 5: 0.7515304860379812
Iteration 3 of the

In [9]:
results['stats_df']

,Generation,Fitness_Mean,Fitness_Std
0,0,0.682756,0.009102
1,1,0.705367,0.019474
2,2,0.708551,0.024225
3,3,0.716312,0.024964
4,4,0.732247,0.022196


----------------------------------------------------------------

In [ ]:
best_solutions_test1 = []
fitness_history_test1 = []

for i in range(NUMBER_OF_TESTS):

    print(f'Iteration {i} of the genetic algorithm\n')
    
    best_solution, fitness_history = genetic_algorithm(
        initial_population=initial_population,
        max_gen=MAX_GEN,
        selection_algorithm=SELECTION_ALGORITHM,
        k_tournment_selection=K_TOURNEMENT_SELECTION,
        maximization = True,
        xo_prob = XO_PROB,
        mut_prob = MUT_PROB,
        mut_max_window_size=MUT_MAX_WINDOW_SIZE,
        elitism = True,
        verbose = False,
    )

    best_solutions_test1.append(best_solution)
    fitness_history_test1.append(best_solution.fitness())


Final best individual in generation 1: 0.6838529241097527
Final best individual in generation 2: 0.6838529241097527
Final best individual in generation 3: 0.6915038610725596
Final best individual in generation 4: 0.6915038610725596
Final best individual in generation 5: 0.7201295581753574
Final best individual in generation 6: 0.7201295581753574
Final best individual in generation 7: 0.7216017595633901
Final best individual in generation 8: 0.7216017595633901
Final best individual in generation 9: 0.7216017595633901
Final best individual in generation 10: 0.741729835362431
Final best individual in generation 1: 0.6763154244898667
Final best individual in generation 2: 0.6948754898044959
Final best individual in generation 3: 0.6948754898044959
Final best individual in generation 4: 0.6948754898044959
Final best individual in generation 5: 0.6948754898044959
Final best individual in generation 6: 0.6948754898044959
Final best individual in generation 7: 0.6948754898044959
Final best ind

KeyboardInterrupt: 

### 2nd test:

....

### 3rd test:


### 4th test:


# 3. Testing Hill Climbing

# 4. Testing Simulated Annealing